In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import pycountry
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm

In [2]:
df=pd.read_csv('Arab-Spring-Paper/Once more/final.csv')
df['Text']=df['Title']+' '+df['Abstract']
df['union_annotation'] = df['union_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df['intersection_annotation'] = df['intersection_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [4]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Function to extract locations and filter by country names
def extract_and_filter_locations(text):
    ner_results = nlp(text)
    locations = [
        entity["word"]
        for entity in ner_results
         if ( entity["entity"] == "B-LOC" or entity["entity"] == "I-LOC")
    ]

    final=coco.convert(list(set(locations)), to='ISO3') 
    if type(final)==str:
        final=[final]

    final=[x.lower() for x in final if x!='not found']
   
    return final

# Apply the function to the DataFrame with tqdm
tqdm.pandas()  # Initialize tqdm for pandas
import logging
import country_converter as coco
coco_logger = coco.logging.getLogger()
coco_logger.setLevel(logging.CRITICAL)
df["locations"] = df["Text"].progress_apply(extract_and_filter_locations)

# Display the result


Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:23:25<00:00,  5.01s/it]


In [5]:
df.to_csv('Transformer/transformer3.csv', index=False)

In [8]:
print('overall accuracy (union)', sum(df['locations']==df['union_annotation'])/1000)
print('overall accuracy (intersection)', sum(df['locations']==df['intersection_annotation'])/1000)

overall accuracy (union) 0.744
overall accuracy (intersection) 0.73


In [9]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "accuracy_union": (x.apply(lambda row: set(row['locations']) == set(row['union_annotation']), axis=1).mean()),
        "accuracy_intersection": (x.apply(lambda row: set(row['locations']) == set(row['intersection_annotation']), axis=1).mean())
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='accuracy_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_19788\2343847047.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,accuracy_union,accuracy_intersection
0,with_mention,0.628571,0.575000
1,with_mention_arab,0.640000,0.595000
2,field_20,0.907692,0.909615


In [10]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "jaccard_union": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['union_annotation']))) /
                            len(set(row['locations']).union(set(row['union_annotation'])))
                if len(set(row['locations']).union(set(row['union_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        ),
        "jaccard_intersection": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['intersection_annotation']))) /
                            len(set(row['locations']).union(set(row['intersection_annotation'])))
                if len(set(row['locations']).union(set(row['intersection_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='jaccard_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_19788\388812476.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,jaccard_union,jaccard_intersection
0,with_mention,0.717619,0.670655
1,with_mention_arab,0.794153,0.735152
2,field_20,0.921538,0.920256


In [15]:
sample_recalls = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "recall_union": (
            x.apply(
                lambda row: len(set(row['locations']) & set(row['union_annotation'])) / len(set(row['union_annotation'])) 
                if len(set(row['union_annotation'])) > 0 else 1,
                axis=1
            ).mean()
        ),
        "recall_intersection": (
            x.apply(
                lambda row: len(set(row['locations']) & set(row['intersection_annotation'])) / len(set(row['intersection_annotation'])) 
                if len(set(row['intersection_annotation'])) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_recalls = sample_recalls.sort_values(by='recall_union').reset_index(drop=True)
sample_recalls


C:\Users\Yasaman\AppData\Local\Temp\ipykernel_19788\3864181111.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_recalls = df.groupby("SampleGroup").apply(


,SampleGroup,recall_union,recall_intersection
0,with_mention,0.796726,0.854107
1,with_mention_arab,0.937208,0.964154
2,field_20,0.946154,0.969231


In [19]:
field_20 = df.query("SampleGroup == 'field_20'")

# Compute TP, TN, FP, FN for each row
metrics = field_20.apply(lambda row: pd.Series({
    "TP": int(set(row['locations']) == set(row['union_annotation']) and len(row['union_annotation']) > 0),
    "TN": int(len(row['union_annotation']) == 0 and len(row['locations']) == 0),
    "FP": int(len(row['union_annotation']) == 0 and len(row['locations']) > 0),
    "FN": int(len(row['union_annotation']) > 0 and len(row['locations']) == 0)
}), axis=1)

# Summarize metrics
total_metrics = metrics.sum()

# Calculate precision, recall, and F1 score
precision = total_metrics["TP"] / (total_metrics["TP"] + total_metrics["FP"]) if (total_metrics["TP"] + total_metrics["FP"]) > 0 else 0
recall = total_metrics["TP"] / (total_metrics["TP"] + total_metrics["FN"]) if (total_metrics["TP"] + total_metrics["FN"]) > 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Output results
{
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1_score
}

{'Precision': 0.8852459016393442,
 'Recall': 0.6835443037974683,
 'F1 Score': 0.7714285714285715}

In [20]:
sample_recalls = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "recall_union": (
            x.apply(
                lambda row: pd.Series({
                    "TP": int(set(row['locations']) == set(row['union_annotation']) and len(row['union_annotation']) > 0),
                    "FN": int(len(row['union_annotation']) > 0 and len(row['locations']) == 0)
                }),
                axis=1
            ).sum(axis=0)
        )
    }).apply(
        lambda metrics: metrics["TP"] / (metrics["TP"] + metrics["FN"]) if (metrics["TP"] + metrics["FN"]) > 0 else 0
    )
).reset_index()

sample_recalls = sample_recalls.sort_values(by='recall_union').reset_index(drop=True)
sample_recalls


C:\Users\Yasaman\AppData\Local\Temp\ipykernel_19788\2643262712.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_recalls = df.groupby("SampleGroup").apply(


,SampleGroup,recall_union
0,field_20,0.683544
1,with_mention,0.797235
2,with_mention_arab,0.944444
